<a href="https://colab.research.google.com/github/thazsobral/plantation-analysis/blob/main/plantation_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Comece por aqui

In [ ]:
# Instalando as bibliotecas necessárias
!pip install rasterio geopandas fiona

In [ ]:
# Importando bibliotecas
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from rasterio.transform import from_origin
import geopandas as gpd
from shapely.geometry import Polygon
import cv2

In [ ]:
# Função para calcular o índice NDVI com suporte a imagens multiespectrais
def calculate_ndvi_multispectral(image_path):
    with rasterio.open(image_path) as src:
        bands = src.read()  # Lê todas as bandas
        nir = bands[3]     # Banda 4 (NIR)
        red = bands[2]     # Banda 3 (Red)
        resolution = src.transform[0]  # Resolução espacial (tamanho do pixel em metros)
        transform = src.transform       # Transformação geoespacial
        crs = src.crs                   # Sistema de referência de coordenadas

    nir = nir.astype(np.float32)
    red = red.astype(np.float32)

    ndvi = (nir - red) / (nir + red + 1e-6)  # Evita divisão por zero
    ndvi_normalized = (ndvi - np.min(ndvi)) / (np.max(ndvi) - np.min(ndvi))

    return ndvi, ndvi_normalized, resolution, transform, crs

In [ ]:
# Função para classificar maturação
def classify_maturation(ndvi):
    maturation_map = np.zeros((*ndvi.shape, 3), dtype=np.uint8)
    binary_classes = {
        "baixa": (ndvi >= 0.0) & (ndvi < 0.3),
        "intermediaria": (ndvi >= 0.3) & (ndvi < 0.6),
        "pronta": (ndvi >= 0.6)
    }

    maturation_map[binary_classes["baixa"]] = [255, 0, 0]
    maturation_map[binary_classes["intermediaria"]] = [255, 255, 0]
    maturation_map[binary_classes["pronta"]] = [0, 255, 0]

    return maturation_map, binary_classes

In [ ]:
# Função para calcular a área total por classe
def calculate_area_by_class(binary_classes, resolution):
    pixel_area = resolution ** 2
    areas = {}

    for class_name, binary_mask in binary_classes.items():
        class_pixels = np.sum(binary_mask)
        areas[class_name] = class_pixels * pixel_area

    return areas

In [ ]:
# Função para salvar o mapa de maturação como GeoTIFF
def save_maturation_map_as_geotiff(binary_classes, transform, crs, output_path):
    with rasterio.open(
        output_path,
        "w",
        driver="GTiff",
        height=binary_classes["baixa"].shape[0],
        width=binary_classes["baixa"].shape[1],
        count=3,  # 3 canais (R, G, B)
        dtype="uint8",
        crs=crs,
        transform=transform,
    ) as dst:
        # Criando canais do mapa de maturação
        dst.write((binary_classes["baixa"] * 255).astype(np.uint8), 1)  # Banda vermelha
        dst.write((binary_classes["intermediaria"] * 255).astype(np.uint8), 2)  # Banda verde
        dst.write((binary_classes["pronta"] * 255).astype(np.uint8), 3)  # Banda azul

In [ ]:
# Função para salvar métricas em um arquivo de texto
def save_metrics(areas, output_path):
    with open(output_path, "w") as f:
        f.write("Áreas por classe de maturação (em m²):\n")
        for class_name, area in areas.items():
            f.write(f" - {class_name.capitalize()}: {area:.2f} m²\n")

In [ ]:
# Carregando uma imagem multiespectral
from google.colab import files
uploaded = files.upload()

In [ ]:
# Lendo o caminho da imagem carregada
image_path = list(uploaded.keys())[0]

In [ ]:
image = cv2.imread(image_path)

In [ ]:
# Calculando o NDVI para a imagem multiespectral
ndvi, ndvi_normalized, resolution, transform, crs = calculate_ndvi_multispectral(image_path)

In [ ]:
# Classificando a maturação
maturation_map, binary_classes = classify_maturation(ndvi_normalized)

In [ ]:
# Calculando a área total por classe
areas = calculate_area_by_class(binary_classes, resolution)

In [ ]:
# Salvando os resultados
save_maturation_map_as_geotiff(binary_classes, transform, crs, "maturation_map.tif")
save_metrics(areas, "areas_maturation.txt")

print("Resultados salvos com sucesso:")
print("- Mapa de maturação salvo como GeoTIFF (maturation_map.tif)")
print("- Áreas por classe salvas em texto (areas_maturation.txt)")

In [ ]:
# Visualizando os resultados
plt.figure(figsize=(15, 10))

In [ ]:
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
# Imagem original
plt.subplot(1, 2, 1)
plt.title("Imagem Original")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")

In [ ]:
# NDVI
plt.subplot(1, 2, 1)
plt.title("Mapa NDVI")
plt.imshow(ndvi_normalized, cmap="Greens")
plt.axis("off")

In [ ]:
# Mapa de maturação
plt.subplot(1, 2, 2)
plt.title("Classificação de Maturação")
plt.imshow(maturation_map)
plt.axis("off")

In [ ]:
from matplotlib.patches import Patch  # Importação da classe Patch
# Adicionando legenda
legend_patches = [
    Patch(color='red', label='Baixa Maturação'),
    Patch(color='yellow', label='Maturação Intermediária'),
    Patch(color='green', label='Pronta para Colheita')
]
plt.legend(handles=legend_patches, loc='lower right', fontsize='medium', frameon=True)

In [ ]:
plt.show()